In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [5]:
import os
from utils import get_openai_api_key, pretty_print_result

from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [6]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [8]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [9]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [10]:
from crewai_tools import BaseTool

In [11]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [12]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [13]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [14]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [15]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

In [16]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for our sales team.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_company_background.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './instructions/DeepLearningAI_company_background.txt'.
 T

In [17]:
from IPython.display import Markdown
Markdown(result)

Email Draft to Andrew Ng:

Subject: Collaboration Opportunity: Aligning our Solutions with DeepLearningAI's Recent Product Launch

Dear Andrew Ng,

I hope this message finds you well. I am reaching out to you on behalf of CrewAI, a company dedicated to providing innovative solutions that align perfectly with the cutting-edge work that DeepLearningAI is known for.

I recently came across the exciting news about DeepLearningAI's recent product launch, particularly the announcement of the Generative AI for Everyone course in collaboration with Coursera. This initiative truly reflects your commitment to democratizing AI knowledge and providing accessible learning opportunities to a global audience.

At CrewAI, we share a similar vision of empowering individuals through AI education and hands-on training. Our solutions are designed to enhance the learning experience and support the growth of AI enthusiasts at all levels. We believe that by combining our expertise with DeepLearningAI's impactful initiatives, we can create a powerful synergy that benefits aspiring AI professionals worldwide.

I am confident that our tailored solutions can complement DeepLearningAI's mission and contribute to the success of your educational programs. Your dedication to advancing AI technologies aligns perfectly with our commitment to providing cutting-edge tools and resources that enable learners to thrive in the ever-evolving field of artificial intelligence.

I would love the opportunity to discuss further how CrewAI can support DeepLearningAI's goals and initiatives. Your leadership and vision have inspired many in the AI community, and we are eager to explore potential collaboration opportunities that drive innovation and excellence.

Thank you for considering this partnership opportunity, Andrew. I look forward to the possibility of working together to make a meaningful impact on the future of AI education.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---

Email Draft to Ted Greenwald:

Subject: Enhancing DeepLearningAI's Strategic Vision with CrewAI's Solutions

Dear Ted Greenwald,

I trust this message finds you well. As the Chief Strategy Officer and Senior Vice President of AI at DeepLearningAI, your strategic insights have undoubtedly played a significant role in shaping the company's success.

I am reaching out to you from CrewAI, a company that specializes in providing tailored solutions to enhance the learning experience and support the growth of AI enthusiasts. Our commitment to innovation and excellence aligns with DeepLearningAI's mission to make world-class AI education accessible to a global audience.

I recently learned about DeepLearningAI's recent product launch, including the introduction of the Generative AI for Everyone course in collaboration with Coursera. This initiative reflects the company's dedication to advancing AI technologies and providing hands-on training opportunities to learners worldwide.

At CrewAI, we are excited about the opportunity to collaborate with DeepLearningAI and contribute to the success of your educational programs. Our solutions are designed to complement your strategic vision and empower individuals to build successful careers in AI.

I believe that by leveraging our expertise and resources, we can further enhance DeepLearningAI's impact in the AI education sector. Your leadership and strategic acumen are truly commendable, and we are eager to explore how CrewAI's solutions can support DeepLearningAI's goals and initiatives.

I would be grateful for the opportunity to discuss potential collaboration opportunities with you. Together, we can drive innovation, inspire learning, and shape the future of AI education for generations to come.

Thank you for considering this partnership, Ted. I look forward to the possibility of working together to achieve shared goals and objectives.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---